In [1]:
import dill as pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

root_dir = '/ihme/homes/rmbarber/covid-19/elastispliner/holdouts'


In [2]:
hierarchy = pd.read_csv('/ihme/covid-19/model-inputs/best/locations/modeling_hierarchy.csv')
is_us = hierarchy['path_to_top_parent'].apply(lambda x: '102' in x.split(','))
is_md = hierarchy['most_detailed'] == 1
hierarchy = hierarchy.loc[is_us & is_md]


In [29]:
def get_change(root_dir, n_holdout_days, location_id):
    with open(f'{root_dir}/2020_06_16.{(n_holdout_days+1):02d}/model_working_dir/model_outputs/{location_id}.pkl', 'rb') as fopen:
        data = pickle.load(fopen)
    model_data = data['model_data']
    date = model_data.loc[~model_data['Death rate'].isnull(), 'Date'].max()
    day = date.strftime('%A')
    prediction = data['smooth_draws'].sort_values(['location_id', 'date']).set_index(['location_id', 'date'])
    prediction = prediction.mean(axis=1)
    prediction.name = 'prediction'
    prediction = prediction.reset_index()
    prediction = prediction.loc[prediction['date'] >= date]
    
    # change in daily deaths over prediction inverval
    prediction = np.diff(np.diff(prediction['prediction'])).sum()
    
    return pd.DataFrame({'location_id':location_id,
                         'date':date,
                         'day':day,
                         'change':prediction}, index=[0])


In [30]:
dfs = []
for location_id in hierarchy['location_id'].to_list():
    if not location_id in [544, 550]:
        for n_holdout_days in range(13):
            df = get_change(root_dir, n_holdout_days, location_id)
            dfs.append(df)
df = pd.concat(dfs)


In [41]:
# for location_id in hierarchy['location_id'].unique():
#     if not location_id in [544, 550]:
#         plt.plot(df.loc[df['location_id'] == location_id, 'date'],
#                  df.loc[df['location_id'] == location_id, 'change'])
#         for sunday in df.loc[(df['location_id'] == location_id) & (df['day'] == 'Sunday'), 'date'].values:
#             plt.axvline(sunday, color='red', linestyle='--')
#         plt.title(hierarchy.loc[hierarchy['location_id'] == location_id, 'location_name'].item())
#         plt.xticks(rotation=60)
#         plt.show()


,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0,0
0,0,0,0,1,0,0,0
0,0,0,1,0,0,0,0
0,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...
0,0,0,0,1,0,0,0
0,0,0,1,0,0,0,0
0,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0
